# Intersubject Correlations
*Written by Luke Chang and Juha Lahnakoski*



In [ ]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from nltools.data import Brain_Data, Design_Matrix, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import regress, zscore, fdr, threshold
from nltools.file_reader import onsets_to_dm
from nltools.external import glover_hrf
from nilearn.plotting import view_img, glass_brain, plot_stat_map
from sklearn.metrics import pairwise_distances
from scipy.stats import binom, ttest_1samp


data_dir = '/Volumes/Engram/Data/Sherlock/fmriprep'


# Extract Average ROI activity


We have already done this step for you, but want to show you how to extract average activity within a set of ROIs from a functional parcellation. There are many ways to do this, but here we will write out a separate CSV for each participant's run.

In [ ]:
mask = Brain_Data('http://neurovault.org/media/images/2099/Neurosynth%20Parcellation_0.nii.gz')

for scan in ['Part1', 'Part2']:
    file_list = glob.glob(os.path.join(data_dir, '*', 'func', f'*crop*{scan}*hdf5'))
    for f in file_list:
        sub = os.path.basename(f).split('_')[0]
        print(sub)
        data = Brain_Data(f)
        roi = data.extract_roi(mask)
        pd.DataFrame(roi.T).to_csv(os.path.join(os.path.dirname(f), f"{sub}_{scan}_Average_ROI_n50.csv" ), index=False)

In [ ]:
base_dir = '/Users/lukechang/Dropbox/Dartbrains/data/Sherlock'

sub_list = [os.path.basename(x).split('_')[0] for x in glob.glob(os.path.join(base_dir,'MovieROI','*Part1*csv'))]
sub_list.sort()

sub_timeseries = {}
for sub in sub_list:
    part1 = pd.read_csv(os.path.join(base_dir, 'MovieROI', f'{sub}_task-sherlockPart1_ROI_n50.csv'))
    part2 = pd.read_csv(os.path.join(base_dir, 'MovieROI', f'{sub}_task-sherlockPart2_ROI_n50.csv'))
    sub_data = part1.append(part2)
#     sub_data = part2
    sub_data.reset_index(inplace=True, drop=True)
    sub_timeseries[sub] = sub_data

# Compute Similarity of Time Courses

## Average Pairwise Similarity Matrix

In [ ]:
isc = {}
for roi in range(50):
    sub_rois = {}
    for sub in sub_timeseries:
        sub_rois[sub] = sub_timeseries[sub].loc[roi]
    sub_rois = pd.DataFrame(sub_rois)
    time_dist = Adjacency(1 - pairwise_distances(sub_rois.T, metric='correlation'), matrix_type='similarity')
    isc[roi] = time_dist.mean()
isc_brain = roi_to_brain(pd.Series(isc), mask_x)


In [ ]:

isrsa_r = Brain_Data(os.path.join(base_dir, 'analyses', 'ISRSA_Scene_Memory_scene_accuracy_r.nii.gz'))
isrsa_p = Brain_Data(os.path.join(base_dir, 'analyses', 'ISRSA_Scene_Memory_scene_accuracy_p.nii.gz'))

threshold(isrsa_r, isrsa_p, thr=.005).plot()

## Similarity with Average Response

# Phase Synchrony

# Recommended reading

* Haxby, J. V., Guntupalli, J. S., Nastase, S. A., & Feilong, M. (2020). Hyperalignment: Modeling shared information encoded in idiosyncratic cortical topographies. Elife, 9, e56601.


* Cohen, J. D., Daw, N., Engelhardt, B., Hasson, U., Li, K., Niv, Y., ... & Willke, T. L. (2017). Computational approaches to fMRI analysis. Nature neuroscience, 20(3), 304.


* Chen, P. H. C., Chen, J., Yeshurun, Y., Hasson, U., Haxby, J., & Ramadge, P. J. (2015). A reduced-dimension fMRI shared response model. In Advances in Neural Information Processing Systems (pp. 460-468).


* Guntupalli, J. S., Hanke, M., Halchenko, Y. O., Connolly, A. C., Ramadge, P. J., & Haxby, J. V. (2016). A model of representational spaces in human cortex. Cerebral cortex, 26(6), 2919-2934.

# Contributions

Luke Chang developed the initial notebook.
